#1. Distributed training with Keras

API provides an abstraction for distributing your training across multiple processing units. The goal is to allow users to enable distributed training using existing models and training code, with minimal changes. 

In [1]:
!pip install -q tf-nightly

%tensorflow_version 2.x
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

     |████████████████████████████████| 519.0MB 31kB/s 
     |████████████████████████████████| 460kB 39.3MB/s 
     |████████████████████████████████| 3.0MB 39.7MB/s 


In [2]:
print(tf.__version__)

2.2.0-dev20200419


##1.1. Download the dataset

Setting with_info to True includes the medadata for the entiredataset, which is being saved here to info. 

In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


##1.2. Define distribution strategy

Create a MirroredStrategy object. This will handle distribution, and provides a context manager to build your model inside.

In [4]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [5]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


##1.3. Setup input pipeline

When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory, and tune the learning rate accordingly. 

In [0]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [0]:
# Pixel values, which are 0-255, have to be normalized to the 0-1 range
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  
  return image, label

In [0]:
# Apply this function to the training and test data. 
# shuffle the training data, and batch it for training. 
# Notice we are alos keeping an in-memory cache of the training data to improve performance.

train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

##1.4. Create the model

Create and compile the Keras model in the context of strategy.scope

In [0]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

##1.5. Define the callbacks

In [0]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [0]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [0]:
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

In [0]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

##1.6. Train and evaluate

In [14]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
  1/938 [..............................] - ETA: 0s - loss: 2.2961 - accuracy: 0.1875WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1271: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


937/938 [============================>.] - ETA: 0s - loss: 0.2095 - accuracy: 0.9388
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 31s 33ms/step - loss: 0.2093 - accuracy: 0.9388
Epoch 2/12
937/938 [============================>.] - ETA: 0s - loss: 0.0658 - accuracy: 0.9806
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 25s 27ms/step - loss: 0.0658 - accuracy: 0.9806
Epoch 3/12
936/938 [============================>.] - ETA: 0s - loss: 0.0460 - accuracy: 0.9862
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 25s 27ms/step - loss: 0.0460 - accuracy: 0.9862
Epoch 4/12
937/938 [============================>.] - ETA: 0s - loss: 0.0251 - accuracy: 0.9929
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 25s 27ms/step - loss: 0.0251 - accuracy: 0.9929
Epoch 5/12
937/938 [============================>.] - ETA: 

In [15]:
# check the checkpoint directory
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index


In [16]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 22ms/step - loss: 0.0358 - accuracy: 0.9882
Eval loss: 0.03579382598400116, Eval Accuracy: 0.9882000088691711


In [22]:
!tensorboard --logdir=/content/logs

Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 289, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 305, in _run_serve_subcommand
    server = self._make_server()
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 409, in _make_server
    self.flags, self.plugin_loaders, self.assets_zip_provider
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/backend/application.py", line 

In [23]:
ls -sh ./logs

total 4.0K
4.0K train/


##1.7. Export to SavedModel

In [24]:
path = 'saved_model/'
model.save(path, save_format='tf')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [25]:
# Load model without strategy.scope
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 22ms/step - loss: 0.0358 - accuracy: 0.9882
Eval loss: 0.03579382598400116, Eval Accuracy: 0.9882000088691711


In [26]:
# Load with strategy.scope
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 3s 22ms/step - loss: 0.0358 - accuracy: 0.9882
Eval loss: 0.03579382598400116, Eval Accuracy: 0.9882000088691711
